# About: Moodleの起動

---

Moodleを起動します。

## 全体構成

![構成](images/moodle-000-01.png)

## 準備

「010-パラメータの設定.ipynb」で指定したパラメータを引き継ぐための準備を行います。

![Moodle起動の準備](images/moodle-020-02.png)

### UnitGroup名の指定

構築環境の UnitGroup名を指定してください。「010-パラメータの設定.ipynb」と同じUnitGroup名を指定することで、既に入力したパラメータを引き継ぐことができます。

In [ ]:
# (例)
# ugroup_name = 'Moodle'

ugroup_name =

次のセルを実行すると「010-パラメータの設定.ipynb」で指定したパラメータを読み込みます。読み込むパラメータの値は、上のセルで指定した UnitGroup名に対応するものになります。UnitGroup名の指定が誤っていると意図したパラメータが読み込めないので注意してください。

In [ ]:
%run scripts/group.py
gvars = load_group_vars(ugroup_name)

group_vars に保存されている値が想定通りであるかチェックします。次のセルを実行してエラーにならないことを確認してください。

> エラーになった場合は `ugroup_name` に指定した値が「010-パラメータの設定.ipynb」に指定した値と同じであることを確認してください。異なる値を指定していた場合は、この節の最初のセル以降をツールバーの
<i class="fa-unfreeze-below-in-section fa"></i>ボタンなどで unfreeze して `ugroup_name` の値を設定し直してください。その後
unfreezeしたセルを全て再実行してください。

In [ ]:
if gvars['project_tag'] != '0':
    raise RuntimeError('ERROR!')

### VCCアクセストークンの入力

VCCにアクセスするためのトークンを入力します。

> VCCのアクセストークンは秘密情報となるためNotebook環境には保存してありません。他の秘密情報については VCC の Vaultサーバに保存してあります。VCCのVaultサーバにアクセスするためにはVCCのアクセストークンが必要となります。そのため、Notebookを新たに実行する度にVCCのアクセストークンを入力する必要があります。

次のセルを実行すると入力枠が表示されるのでアクセストークンの値を入力してください。

> アクセストークン入力後に Enter キーを押すことで入力が完了します。

In [ ]:
from getpass import getpass
vcc_access_token = getpass()

入力されたアクセストークンが正しいことを、実際にVCCにアクセスして確認します。

In [ ]:
from common import logsetting
from vcpsdk.vcpsdk import VcpSDK

vcp = VcpSDK(vcc_access_token)

上のセルの実行結果がエラーとなり以下のようなメッセージが表示されている場合は、入力されたアクセストークンに誤りがあります。

```
config vc failed: http_status(403)
2020/XX/XX XX:XX:XX UTC: VCPAuthException: xxxxxxx:token lookup is failed: permission denied
```

エラーになった場合はこの節のセルを全て `unfreeze` してから、もう一度アクセストークンの入力を行ってください。

### AnsibleからVaultサーバにアクセスするための準備

このNotebookでは、VCノードに環境を構築するために Ansible を用います。「010-パラメータの設定.ipynb」で group_varsファイルに記録したパラメータはAnsibleから直接参照することができます。しかしVCCのVaultサーバに記録したパラメータは、そのままではAnsibleから参照することができません。そこで Ansible から Vaultサーバの値を参照するために必要となるライブラリをインストールします。また VCC の Vaultサーバにアクセスするために必要となるトークンと Vault サーバのアドレスを環境変数に設定します。

まず Vaultサーバにアクセスするために必要となるライブラリのインストールを行います。

In [ ]:
!pip install hvac --user

VCCのVault サーバにアクセスする際に必要となるパラメータを環境変数に設定します。

In [ ]:
import os
os.environ['VAULT_ADDR'] = vcp.vcc_info()['vault_url']
os.environ['VAULT_TOKEN'] = vcc_access_token

## VCディスクの作成

Moodleとデータベース用にそれぞれ専用の仮想ディスクを用意します。

VCP SDK を利用すると、クラウドプロバイダに応じた仮想ディスクを作成することができます。このアプリケーションテンプレートではMoodle、データベース用にそれぞれひとつづつ仮想ディスクを作成します。VCP APIで作成した仮想ディスクのことを VCディスクと呼んでいます。

![VCディスクの作成](images/moodle-020-03.png)

### UnitGroupの作成

VCディスクを管理するための UnitGroup を作成します。

UnitGroupを作成するまえに、現在のUnitGroupの一覧を確認します。

In [ ]:
vcp.df_ugroups()

UnitGroupを作成します。

In [ ]:
ug_disk = vcp.create_ugroup(ugroup_name + '_disk', ugroup_type='storage')

UnitGroup作成後の一覧を表示させます。

In [ ]:
vcp.df_ugroups()

### VCディスクの作成

Moodle用のVCディスクを作成します。

In [ ]:
moodle_disk_spec = vcp.get_spec(gvars['vc_provider'] + '_disk', 'small')
moodle_disk_spec.size = gvars['moodle_disk_size']
ug_disk.create_unit('moodle_disk', moodle_disk_spec)

データベース用のVCディスクを作成します。

In [ ]:
db_disk_spec = vcp.get_spec(gvars['vc_provider'] + '_disk', 'small')
db_disk_spec.size = gvars['db_disk_size']
ug_disk.create_unit('db_disk', db_disk_spec)

作成したVCディスクの一覧を表示します。

In [ ]:
ug_disk.df_nodes()

## VCノードの起動

VCP SDK を利用してVCノードを起動します。

![VCディスクの作成](images/moodle-020-04.png)

### VCノードの spec を指定する

「010-パラメータの設定.ipynb」で指定したパラメータをVCノードの `spec` に設定します。

`spec`に設定するパラメータを以下に示します。

* `images`: Baseコンテナイメージ
  - Baseコンテナイメージを設定します
* `params_v`: ボリューム設定
  - Baseコンテナのボリュームを設定します
  - Moodle用Baseコンテナではホスト側の `/sys/fs/cgroup`, `/lib/modules` をコンテナから見えるように設定する必要があります
* `ip_addresses`: IPアドレス
  - VCノードに割り当てるプライベートIPアドレスを設定します
* `disks`: VCディスクのリスト
  - VCノードにアタッチするVCディスクのリストを設定します
* `set_ssh_publickey()`: SSHの公開鍵
  - VCノードに登録するSSHの公開鍵

In [ ]:
import os

vc_provider = gvars['vc_provider']
spec = vcp.get_spec(vc_provider, gvars['vc_flavor'])

# Baseコンテナイメージを指定する
spec.image = 'harbor.vcloud.nii.ac.jp/vcp/moodle:1.6-base'
spec.params_v = [
    '/sys/fs/cgroup:/sys/fs/cgroup:ro',
    '/lib/modules:/lib/modules:ro',
]

# VCノードに割り当てるIPアドレスを指定する
spec.ip_addresses = [gvars['vc_moodle_ipaddress']]

# VCノードにアタッチするVCディスクを指定する
spec.disks = ug_disk.find_nodes()

# VCノードにsshでログインするための公開鍵を指定する
spec.set_ssh_pubkey(os.path.expanduser(gvars['ssh_public_key_path']))

# インスタンスタイプの指定
if 'vc_vm_type' in gvars:
    if vc_provider == 'aws':
        spec.instance_type = gvars['vc_vm_type']
    elif vc_provider == 'azure':
        spec.vm_size = gvars['vc_vm_type']

`spec` の設定値を確認します。

In [ ]:
print(spec)

### VCノードの起動

UnitGroupを作成します。

In [ ]:
ugroup = vcp.create_ugroup(ugroup_name)

VCノードを起動します。

> 起動には1分半程度の時間がかかります。

In [ ]:
unit = ugroup.create_unit('moodle', spec)

VCノードの状態が `RUNNING` になっていることを確認します。

> VCノードの起動に失敗して`RUNNING`以外の状態になっている場合は次のセルを実行するとエラーになります。エラーになった場合は、`ugroup.cleanup()` を実行して VCノードを削除してください。

In [ ]:
if any([node.state != 'RUNNING' for node in unit.find_nodes()]):
    raise RuntimeError('ERROR: not running')

起動したVCノードの一覧を表示します。

In [ ]:
ugroup.df_nodes()

## Ansibleの設定

VCノードをAnsibleで操作するための設定を行います。

![Ansibleの設定](images/moodle-020-05.png)

まず、VCノードにSSHでログインできるようにするために `~/.ssh/known_hosts` の更新を行います。

> 何度かVCノードの起動を行うと、異なるホストが同じIPアドレスで起動するためにSSHのホストキーのチェックでエラーになる事があります。このような状況に対応するために、起動したVCノードのIPアドレスに対応するエントリを`known_hosts`ファイルから削除します。その後、`ssh-keyscan`コマンドを利用して起動したVCノードのホストキーを取得して `known_hosts`ファイルの内容を更新します。

In [ ]:
from time import sleep

def check_update_known_hosts(ipaddr):
    # VCノード起動直後だと sshd サービスが開始されておらずに known_hosts が更新されない場合がある
    # ssh-keyscan が値を取得できるまで何度かリトライする
    for x in range(10):
        out = ! echo $(ssh-keyscan {ipaddr} 2> /dev/null | wc -l)
        update_lines = int(out[0])
        if update_lines > 0:
            break
        sleep(1)
    else:
        raise RuntimeError("ERROR: timeout!")    

!mkdir -p -m 0700 ~/.ssh
!touch ~/.ssh/known_hosts
for addr in ugroup.find_ip_addresses():
    !ssh-keygen -R {addr}
    check_update_known_hosts(addr)
    !ssh-keyscan -H {addr} >> ~/.ssh/known_hosts

起動したVCノードに対応するエントリを Ansible のインベントリに登録します。

> Ansibleで操作を行うためには、操作対象のホスト(IPアドレス)をインベントリに登録する必要があります。

In [ ]:
import yaml

inventory = {'all': {'children': {
    ugroup.name: {
        'hosts': dict([(ip, {}) for ip in ugroup.find_ip_addresses()]),
        'vars': {
            'ansible_user': 'vcp',
            'ansible_ssh_private_key_file': gvars['ssh_private_key_path'],
            'ansible_python_interpreter': '/usr/bin/python',
            
        }
    }
}}}

with open('inventory.yml', 'w') as f:
    yaml.safe_dump(inventory, f, default_flow_style=False)
    
!cat inventory.yml

先程VCノードを登録したファイルをインベントリとして指定するためのAnsibleのコンフィギュレーションファイルを作成します。

> カレントディレクトリにコンフィギュレーションファイル(`ansible.cfg`)を作成すると、Ansibleを実行する際にその設定が適用されます。

In [ ]:
from pathlib import Path

inventory_path = Path('./inventory.yml').resolve()
ansible_cfg = Path('./ansible.cfg').resolve()
with ansible_cfg.open(mode='w') as f:
    f.write(f'''
[defaults]
inventory = {inventory_path}
command_warnings = False
''')
ansible_cfg.parent.chmod(0o755)

!cat ./ansible.cfg

VCノードに対して Ansible で接続できることを確認します。

In [ ]:
!ansible {ugroup_name} -m ping

正常に接続できると以下のように表示されます。

```
XXX.XXX.XXX.XXX | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
```

VCノードに対して設定ファイルの変更やパッケージの追加を行う場合にVCノードの管理者権限が必要になる場合があります。Ansibleで管理者権限によるコマンド実行が可能かどうかを確認します。

In [ ]:
# 管理者権限(-b)でのコマンド実行
!ansible {ugroup_name} -b -a 'whoami'

## データ格納領域の準備

Moodle用, データベース用のディスクをデータを格納する領域として利用できるように準備します。

![データ格納領域の準備](images/moodle-020-06.png)

### デバイス名の指定

Moodle用、データベース用、それぞれのディスクに対応するデバイス名をここで指定します。

まず、VCノードのブロックデバイスの一覧を確認します。

In [ ]:
!ansible {ugroup_name} -a 'lsblk'

データベース用、Moodle用ディスクに対応するブロックデバイス名を以下のセルで指定してください。

> ブロックデバイス名はクラウドプロバイダやインスタンスタイプによって異なる値になるので、上のセルの出力結果を確認して値を設定してください。

Moodle用ディスクのデバイス名を指定してください。

In [ ]:
# (例)
# moodle_disk_device = '/dev/xvdf'    # AWS
# moodle_disk_device = '/dev/nvme2n1' # AWS(Nitroベース)
# moodle_disk_device = '/dev/sdc'     # Azure

moodle_disk_device =

データベース用ディスクのデバイス名を指定してください。

In [ ]:
# (例)
# db_disk_device = '/dev/xvdg'    # AWS
# db_disk_device = '/dev/nvme1n1' # AWS(Nitroベース)
# db_disk_device = '/dev/sdd'     # Azure

db_disk_device =

### 論理ボリューム(LV)の作成

LVMを用いて、各用途に応じた論理ボリュームを作成します。

ここでは以下の論理ボリュームを作成します。

* Moodle用ディスク
  - Moodleデータ用論理ボリューム
    - `moodledata` ディレクトリ用の論理ボリューム
  - PHP用論理ボリューム
    - MoodleのPHPファイルを格納する論理ボリューム
* データベース用ディスク
  - データベース用論理ボリューム
  
ここで作成する論理ボリュームは全てシンプロビジョニングされたものとします。そのため、すべての論理ボリュームの容量は、実際のディスク容量に依存しない、仮想化した値を設定できます。また、パラメータ設定の際にボリュームの暗号化を行うように設定した場合は、作成した各論理ボリュームに対して[LUKS](https://gitlab.com/cryptsetup/cryptsetup)を用いた暗号化を行います。

#### Moodle用ボリュームの作成

まずMoodle用ボリュームのボリュームグループ(VG)を作成します。VGの名前を次のセルで指定します。

In [ ]:
moodle_vg_name = 'moodle'

VGを作成します。

In [ ]:
!ansible {ugroup_name} -b -m lvg -a 'pvs={moodle_disk_device} vg={moodle_vg_name}'

VGが作成されたことを確認するために一覧を表示してみます。

In [ ]:
!ansible {ugroup_name} -b -a 'vgs'

作成したVGに thin poolを作成します。プールの名前を次のセルで指定します。

In [ ]:
moodle_thinpool_name = 'moodle-pool'

プールを作成します。

In [ ]:
!ansible {ugroup_name} -b -m lvol -a \
    'vg={moodle_vg_name} thinpool={moodle_thinpool_name} size=98%VG'

プールが作成されたことを確認するために一覧を表示してみます。

In [ ]:
!ansible {ugroup_name} -b -a 'lvs -S "lv_attr=~^t.+"'

作成したプールにmoodledata用のシンプロビジョニングボリュームを作成します。

In [ ]:
moodle_lv_name = f'{gvars["project_tag"]}_moodle'
moodle_lv_size = f'{gvars["moodle_volume_data_size"]}g'

!ansible {ugroup_name} -b -m lvol -a \
    'vg={moodle_vg_name} thinpool={moodle_thinpool_name} \
    lv={moodle_lv_name} size={moodle_lv_size}'

作成した論理ボリュームの情報を表示してみます。

In [ ]:
!ansible {ugroup_name} -b -a \
    'lvdisplay {moodle_vg_name}/{moodle_lv_name}'

パラメータ設定でボリュームの暗号化を有効にした場合は[LUKS](https://gitlab.com/cryptsetup/cryptsetup)による暗号化を行います。

ボリュームを暗号化する手順を直接記述するのは煩雑となるので Ansible の playbook に記述してあります。
playbookで実行している主な手順を以下に示します。

1. VCCのVaultから登録してある暗号化キーを取得する
1. `cryptsetup luksFormat`を実行し暗号化ストレージとして扱えるようにセットアップする
1. `cryptsetup open` を実行し暗号化ストレージを読み書き可能な状態にする

In [ ]:
!ansible-playbook -l {ugroup_name} playbooks/encrypt-moodle_data.yml

プールにMoodleのPHPファイルを格納するシンプロビジョニングボリュームを作成します。

In [ ]:
moodle_php_lv_name = f'{gvars["project_tag"]}_php'
moodle_php_lv_size = f'{gvars["moodle_volume_php_size"]}g'

!ansible {ugroup_name} -b -m lvol -a \
    'vg={moodle_vg_name} thinpool={moodle_thinpool_name} \
    lv={moodle_php_lv_name} size={moodle_php_lv_size}'

作成した論理ボリュームの情報を表示してみます。

In [ ]:
!ansible {ugroup_name} -b -a \
    'lvdisplay {moodle_vg_name}/{moodle_php_lv_name}'

パラメータ設定でボリュームの暗号化を有効にした場合は[LUKS](https://gitlab.com/cryptsetup/cryptsetup)による暗号化を行います。

In [ ]:
!ansible-playbook -l {ugroup_name} playbooks/encrypt-moodle_php.yml

#### データベース用ボリュームの作成

まずデータベース用ボリュームのボリュームグループ(VG)を作成します。VGの名前を次のセルで指定します。

In [ ]:
db_vg_name = 'db'

VGを作成します。

In [ ]:
!ansible {ugroup_name} -b -m lvg -a 'pvs={db_disk_device} vg={db_vg_name}'

作成したVGに thin poolを作成します。プールの名前を次のセルで指定します。

In [ ]:
db_thinpool_name = 'db-pool'

プールを作成します。

In [ ]:
!ansible {ugroup_name} -b -m lvol -a \
    'vg={db_vg_name} thinpool={db_thinpool_name} size=98%VG'

作成したプールにシンプロビジョニングボリュームを作成します。

In [ ]:
db_lv_name = f'{gvars["project_tag"]}_db'
db_lv_size = f'{gvars["db_volume_size"]}g'
!ansible {ugroup_name} -b -m lvol -a \
    'vg={db_vg_name} thinpool={db_thinpool_name} \
    lv={db_lv_name} size={db_lv_size}'

作成した論理ボリュームの情報を表示してみます。

In [ ]:
!ansible {ugroup_name} -b -a 'lvdisplay {db_vg_name}/{db_lv_name}'

パラメータ設定でボリュームの暗号化を有効にした場合は[LUKS](https://gitlab.com/cryptsetup/cryptsetup)による暗号化を行います。

In [ ]:
!ansible-playbook -l {ugroup_name} playbooks/encrypt-db.yml

### ファイルシステムの作成

前節で作成した各論理ボリュームにファイルシステムを作成します。使用するファイルシステムは XFS とします。

#### Moodle Data用ボリュームのファイルシステム作成

Moodle Data用ボリュームにファイルシステムを作成します。

In [ ]:
moodle_vol = f'{moodle_vg_name}-{moodle_lv_name}{"-enc" if gvars["moodle_volume_encrypt"] else ""}'
!ansible {ugroup_name} -b -a \
    'mkfs.xfs /dev/mapper/{moodle_vol}'

マウントします。

In [ ]:
!ansible {ugroup_name} -b -m mount -a \
    'fstype=xfs state=mounted \
    name=/opt/moodle/moodle-{{{{project_tag}}}}/data/moodledata \
    src=/dev/mapper/{moodle_vol}'

マウントされたことを確認します。

In [ ]:
!ansible {ugroup_name} -m shell -a \
    'mount | grep /opt/moodle/moodle-{{{{project_tag}}}}/data/moodledata'

#### PHPファイル用ボリュームのファイルシステム作成

ファイルシステムを作成します。

In [ ]:
php_vol = f'{moodle_vg_name}-{moodle_php_lv_name}{"-enc" if gvars["moodle_volume_encrypt"] else ""}'
!ansible {ugroup_name} -b -a \
    'mkfs.xfs /dev/mapper/{php_vol}'

マウントします。

In [ ]:
!ansible {ugroup_name} -b -m mount -a \
    'fstype=xfs state=mounted \
    name=/opt/moodle/moodle-{{{{project_tag}}}}/data/php \
    src=/dev/mapper/{php_vol}'

マウントされたことを確認します。

In [ ]:
!ansible {ugroup_name} -m shell -a \
    'mount | grep /opt/moodle/moodle-{{{{project_tag}}}}/data/php'

#### データベース用ボリュームのファイルシステム作成

データベース用ボリュームにファイルシステムを作成します。

In [ ]:
db_vol = f'{db_vg_name}-{db_lv_name}{"-enc" if gvars["db_volume_encrypt"] else ""}'
!ansible {ugroup_name} -b -a \
    'mkfs.xfs /dev/mapper/{db_vol}'

マウントします。

In [ ]:
!ansible {ugroup_name} -b -m mount -a \
    'fstype=xfs state=mounted \
    name=/opt/moodle/db-{{{{project_tag}}}}/data \
    src=/dev/mapper/{db_vol}'

マウントされたことを確認します。

In [ ]:
!ansible {ugroup_name} -m shell -a \
    'mount | grep /opt/moodle/db-{{{{project_tag}}}}/data'

## 設定ファイルの配置

アプリケーションコンテナを実行するのに必要となる設定ファイルを実行環境(VCノード)に配置します。

![設定ファイルの配置](images/moodle-020-07.png)

### docker-compose

ここで構築するMoodle環境は複数のコンテナで構成されています。`docker-compose`を使うと、`docker-compose.yml` に記述した複数のコンテナを管理することができまる。ここでは `docker-compose.yml` などの設定ファイルをVCノードに配置します。

VCノードに配置するファイルを次の表に示します。

<table>
  <tr>
    <th style="text-align:left;">VCノードのパス</th>
    <th style="text-align:left;">説明</th>
  </tr>
  <tr>
    <td style="text-align:left;">/opt/moodle/docker-compose.yml</td>
    <td style="text-align:left;">docker-composeの構成ファイル</td>
  </tr>
  <tr>
    <td style="text-align:left;">/opt/moodle/.env.db</td>
    <td style="text-align:left;">データベースへの接続情報をコンテナの環境変数に設定するためのファイル</td>
  </tr>
</table>

設定ファイルを格納するVCノードのディレクトリを作成します。

In [ ]:
!ansible {ugroup_name} -b -m file -a 'path=/opt/moodle state=directory owner=vcp'

`docker-compose.yml`をVCノードに配置します。

In [ ]:
!ansible {ugroup_name} -m template -a \
    'src=template/docker/compose/docker-compose.yml \
    dest=/opt/moodle/'

データベースへの接続情報については、直接 `docker-compose.yml` に記さずに別ファイル`.env.db`に記述しています。`.env.db`ファイルをVCノードに配置します。

In [ ]:
!ansible {ugroup_name} -m template -a \
    'src=template/docker/compose/env \
    dest=/opt/moodle/.env.db'

### Moodleコンテナ

Moodleコンテナの設定ファイルをVCノードに配置します。

構築環境では利用者が変更する可能性のある `httpd.conf` などの設定ファイルはVCノードに配置しています。VCノードに配置したファイルはDockerの [bind mount](https://docs.docker.com/storage/bind-mounts/)の機能を利用して、コンテナ側から参照できるように設定しています。このようにすることで、コンテナがステートレスとなるようにしています。

Moodleコンテナが参照している設定ファイルの、コンテナ内のパスと、VCノードにおけるパスの対応関係を以下の表に示します。

<table>
  <tr>
    <th style="text-align:left;">コンテナ内のパス</th>
    <th style="text-align:left;">VCノードのパス</th>
  </tr>
  <tr>
    <td style="text-align:left;">/etc/httpd/conf/httpd.conf</td>
    <td style="text-align:left;">/opt/moodle/moodle-0/conf/httpd/conf/httpd.conf</td>
  </tr>
  <tr>
    <td style="text-align:left;">/etc/httpd/conf.d/</td>
    <td style="text-align:left;">/opt/moodle/moodle-0/conf/httpd/conf.d/</td>
  </tr>
  <tr>
    <td style="text-align:left;">/etc/httpd/conf.modules.d/</td>
    <td style="text-align:left;">/opt/moodle/moodle-0/conf/httpd/conf.modules.d/</td>
  </tr>
  <tr>
    <td style="text-align:left;">/etc/php.ini</td>
    <td style="text-align:left;">/opt/moodle/moodle-0/conf/php.ini</td>
  </tr>
  <tr>
    <td style="text-align:left;">/etc/php.d/</td>
    <td style="text-align:left;">/opt/moodle/moodle-0/conf/php.d/</td>
  </tr>
  <tr>
    <td style="text-align:left;">/etc/php-zts.d/</td>
    <td style="text-align:left;">/opt/moodle/moodle-0/conf/php-zts.d/</td>
  </tr>
</table>

次のセルを実行するとMoodleコンテナに関する設定ファイルをVCノードに配置します。

In [ ]:
moodle_cnt = f'moodle-{gvars["project_tag"]}'
!ansible {ugroup_name} -b -m file -a \
    'path=/opt/moodle/{moodle_cnt} state=directory owner=vcp'
!ansible {ugroup_name} -m synchronize -a \
    'src=template/docker/compose/conf/moodle/conf dest=/opt/moodle/{moodle_cnt}'

### データベースコンテナ

データベースコンテナが参照している設定ファイルの、コンテナ内のパスと、VCノードにおけるパスの対応関係を以下の表に示します。

<table>
  <tr>
    <th style="text-align:left;">コンテナ内のパス</th>
    <th style="text-align:left;">VCノードのパス</th>
  </tr>
  <tr>
    <td style="text-align:left;">/etc/mysql/mariadb.conf.d/</td>
    <td style="text-align:left;">/opt/moodle/db-0/conf/mariadb.conf.d/</td>
  </tr>
</table>

データベースコンテナとして利用している MariaDBコンテナでは、`/etc/mysql/mariadb.conf.d/` に配置した `*.cnf` ファイルを読み込みます。後から読み込んだ設定ファイルの内容で設定値は上書きされるので、上記のディレクトリに設定ファイルを配置することで `/etc/mysql/my.cnf` に記された設定を変更することができます。

### リバースプロキシコンテナ

リバースプロキシとして利用する Apache HTTP Serverコンテナが参照している設定ファイルの、コンテナ内のパスと、VCノードにおけるパスの対応関係を以下の表に示します。

<table>
  <tr>
    <th style="text-align:left;">コンテナ内のパス</th>
    <th style="text-align:left;">VCノードのパス</th>
  </tr>
  <tr>
    <td style="text-align:left;">/usr/local/apache2/conf/</td>
    <td style="text-align:left;">/opt/moodle/proxy/conf/</td>
  </tr>
  <tr>
    <td style="text-align:left;">/usr/local/apache2/conf/httpd.conf</td>
    <td style="text-align:left;">/opt/moodle/proxy/conf/httpd.conf</td>
  </tr>
  <tr>
    <td style="text-align:left;">/usr/local/apache2/conf/moodle-proxy.conf</td>
    <td style="text-align:left;">/opt/moodle/proxy/conf/moodle-proxy.conf</td>
  </tr>
  <tr>
    <td style="text-align:left;">/usr/local/apache2/conf/cert/</td>
    <td style="text-align:left;">/opt/moodle/cert/</td>
  </tr>
</table>

次のセルを実行するとApache HTTP Serverコンテナに関する設定ファイルをVCノードに配置します。

In [ ]:
!ansible {ugroup_name} -m synchronize -a \
    'src=template/docker/compose/conf/proxy/conf dest=/opt/moodle/proxy'

リバースプロキシに関する設定を記したファイルをVCノードに配置します。

In [ ]:
!ansible {ugroup_name} -m template -a \
    'src=template/docker/compose/moodle-proxy.conf.template \
    dest=/opt/moodle/proxy/conf/moodle-proxy.conf'

サーバ証明書をVCノードに配置します。

In [ ]:
!ansible {ugroup_name} -m file -a 'state=directory path=/opt/moodle/cert'
!ansible {ugroup_name} -m copy -a 'src={{{{rproxy_tls_cert_path}}}} dest=/opt/moodle/cert/server.crt'
!ansible {ugroup_name} -m copy -a 'src={{{{rproxy_tls_key_path}}}} dest=/opt/moodle/cert/server.key'

## アプリケーションコンテナの起動

Moodleコンテナ、データベースコンテナ、リバースプロキシコンテナを起動して、Moodle環境を起動します。

![コンテナの起動](images/moodle-020-08.png)

docker-composeコマンドを実行してコンテナの起動を行います。

In [ ]:
!ansible {ugroup_name} -a 'chdir=/opt/moodle docker-compose up -d'

Moodleのインストール処理が完了してMoodleのサービスが開始されるまで数分程度を要します。ここではMoodleのサービスが開始されるまでの待ち合わせを行います。

> Moodleが起動するまでは `503 Service Unavailable` と表示されます。その間は、そのまま待ってください。

In [ ]:
%run scripts/utils.py
moodle_url = 'https://' + gvars['vc_moodle_ipaddress']

def check_not_bad_gateway():
    try:
        !curl -k -s -I {moodle_url} | grep 503
        raise Exception()
    except RuntimeError:
        pass

retry_exec(check_not_bad_gateway, err=Exception)

## Moodle を利用できることを確認

構築したMoodle環境にアクセスし、Moodle が利用できることを確認します。

### アドレスの確認

次のセルを実行すると表示されるリンクが、構築したMoodle環境のアドレスです。リンクをクリックしてMoodle環境にログインしてください。

In [ ]:
from IPython.core.display import HTML
HTML(u'<a href="{0}" target="_blank">{0}</a>'.format(gvars['moodle_url']))

### ライブログの確認

Moodle のライブログを確認します。次のセルを実行すると表示されるリンクをクリックしてください。

In [ ]:
from IPython.core.display import HTML
HTML(u'<a href="{0}/report/loglive/index.php" target="_blank">{0}/report/loglive/index.php</a>'.format(gvars['moodle_url']))